# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

geoapify_key

'6f88fcd0004c457997cfcab176d50ca4'

In [2]:
#pip install geoviews
# found out this and below were already installed, we just needed to restart kernel and rerun cells before doing work on this notebook as the install took place way after the install took place

In [3]:
#pip install cartopy

In [4]:
#ls

In [5]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("city_data_updated.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port-aux-francais,-49.3500,70.2167,3.60,81,100,16.25,TF,1691444815
1,1,grytviken,-54.2811,-36.5092,-6.35,82,67,4.40,GS,1691444815
2,2,sarmiento,-45.5882,-69.0700,6.49,56,100,1.95,AR,1691444815
3,3,afaahiti,-17.7500,-149.2833,27.93,79,97,5.51,PF,1691444815
4,4,taoudenni,22.6783,-3.9836,40.53,17,0,4.44,ML,1691444815


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [6]:
# what does the code below mean/do?
# %%capture --no-display

# Configure the map plot
first_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 1000,
    frame_height = 800,
    size = "Humidity",
    scale = 0.5,
    color = "City"
)

# Display the map
first_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [7]:
# Narrow down cities that fit criteria and drop any results with null values
# nobody could tell me if there was a specific criteria for this cell so i just put in what makes an ideal place for me personally!

ideal_city_data_df = city_data_df.loc[(city_data_df["Humidity"] < 80) & (city_data_df["Max Temp"] > 23) & (city_data_df["Max Temp"] < 30)]

# Drop any rows with null values
ideal_city_data_df.dropna()

# Display sample data
ideal_city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
3,3,afaahiti,-17.7500,-149.2833,27.93,79,97,5.51,PF,1691444815
25,25,sidi aissa,35.8855,3.7724,26.50,43,0,3.31,DZ,1691444818
33,33,whitehorse,60.7161,-135.0538,23.90,41,75,4.63,CA,1691444587
39,39,anna regina,7.2644,-58.5077,28.07,77,22,3.58,GY,1691444820
43,43,lompoc,34.6391,-120.4579,26.03,74,100,3.60,US,1691444821


### Step 3: Create a new DataFrame called `hotel_df`.

In [8]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity

hotel_df = pd.DataFrame({
                "City": ideal_city_data_df["City"],
                "Country": ideal_city_data_df["Country"],
                "Lat": ideal_city_data_df["Lat"],
                "Lng": ideal_city_data_df["Lng"],
                "Humidity": ideal_city_data_df["Humidity"]
#                :
})

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
# Display sample data

hotel_df['Hotel Name'] = ""
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
3,afaahiti,PF,-17.7500,-149.2833,79,
25,sidi aissa,DZ,35.8855,3.7724,43,
33,whitehorse,CA,60.7161,-135.0538,41,
39,anna regina,GY,7.2644,-58.5077,77,
43,lompoc,US,34.6391,-120.4579,74,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [9]:
# Set parameters to search for a hotel
# some of the inspiration: https://apidocs.geoapify.com/docs/place-details/#api

categories = "accommodation.hotel"
radius = 10000

#filters = f"circle:{hotel_df['Lng']},{hotel_df['Lat']},{radius}"
#bias = f"proximity:{hotel_df['Lng']},{hotel_df['Lat']}"
#limit = 20

params = {
    "categories": categories,
#    "filters": filters, being created in the for loop...
#    "bias": bias, being created in the for loop...
#    "limit": limit, can be used here if needed...
    "apiKey": geoapify_key
}

# Set base URL
base_url = "https://api.geoapify.com/v2/places"

# run a request using our params dictionary
response = requests.get(base_url, params=params)
#response.url

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # YOUR CODE HERE
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
#    params["filters"] = f"circle:{hotel_df['Lng']},{hotel_df['Lat']},{radius}"
#    params["bias"] = f"proximity:{hotel_df['Lng']},{hotel_df['Lat']}"

    params["filters"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

#name_address

Starting hotel search
afaahiti - nearest hotel: Omati Lodge
sidi aissa - nearest hotel: No hotel found
whitehorse - nearest hotel: Town & Mountain Hotel
anna regina - nearest hotel: Jaigobin Hotel
lompoc - nearest hotel: Embassy Suites by Hilton Lompoc Central Coast
khemisset - nearest hotel: النجمة الخضراء
bayganin - nearest hotel: No hotel found
avarua - nearest hotel: Paradise Inn
ylojarvi - nearest hotel: Hotelli Lamminpää
az zuwaytinah - nearest hotel: فندق امال ليبيا
brookings - nearest hotel: Quality Inn
betio village - nearest hotel: Seaman Hostel
staryye ozinki - nearest hotel: No hotel found
erenhot - nearest hotel: Oulen hotel
mata-utu - nearest hotel: Gîte Oceania
mahina - nearest hotel: Motu Martin
kiiminki - nearest hotel: Lasaretti
utrik - nearest hotel: No hotel found
sakaiminato - nearest hotel: かぐら旅館
gazli - nearest hotel: Bukhara Desert Oasis & Spa
zhanaozen - nearest hotel: No hotel found
tofol - nearest hotel: Nautilus Resort
barcelos - nearest hotel: Hotel Dom Nun

,City,Country,Lat,Lng,Humidity,Hotel Name
3,afaahiti,PF,-17.7500,-149.2833,79,Omati Lodge
25,sidi aissa,DZ,35.8855,3.7724,43,No hotel found
33,whitehorse,CA,60.7161,-135.0538,41,Town & Mountain Hotel
39,anna regina,GY,7.2644,-58.5077,77,Jaigobin Hotel
43,lompoc,US,34.6391,-120.4579,74,Embassy Suites by Hilton Lompoc Central Coast
...,...,...,...,...,...,...
552,las trojes,HN,14.0667,-85.9833,63,Hotel Yorling E&E
555,beledweyne,SO,4.7358,45.2036,67,Hotel Medina
557,khartsyz'k,UA,48.0424,38.1473,39,"Готель ""Сілур"""
561,kumagunnam,NG,13.1551,10.6342,72,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [11]:
# Configure the map plot
# to only show hotel name and country- https://github.com/holoviz/hvplot/issues/180

hover_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 0.5,
    color = "City",
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
hover_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)